## Some Preliminaries

For the purposes of this exploration of sql, 'structured query language', I have already obtained data about Roman Amphitheatres from archaeologist Sebastian Heath. Heath made his data availabe as a csv file which he put online via Github. I used a command called 'curl' to obtain it:

```bash
!curl https://raw.githubusercontent.com/sfsheath/roman-amphitheaters/master/roman-amphitheaters.csv > amphi.csv
```
'curl' grabs the information at a web url; the `>` tells it to save everything it finds as `amphi.csv`

I also retrieved information about Roman aqueducts from the [Atlas Project of Roman Aquedcuts](https://romaq.org/the-project/aqueducts.html) and formatted it as a csv with some regular expressions work.

Honestly, half the battle in digital archaeology is getting the data in a format where you can _do_ something.

Fortunately, those two files are already present for you. Double click on them in the file tray, have a look, then come back here.

### Get sqlitebiter
We are going to combine those two files into a new sqlite database where each file is its own unique table.

'sql' stands for 'structured query language' and is a powerful way to create, access, and manage large sql-powered databases. 'sqlite' is a lite version of that larger featureset. '[sqlitebiter](https://sqlitebiter.readthedocs.io/en/latest/)' is a utility that we'll use to set up our data as a .db file. 

In [ ]:
## we'll install it to our working python environment
!pip install sqlitebiter

### CSVs -> sqlite.db

First we'll create a new database called `roman.db` from the two .csv files.

Then we'll initialize it, make sure everything looks ok.

In [ ]:
!sqlitebiter -o roman.db file "amphi.csv" "aqua.csv"
!sqlite3 roman.db .schema .exit
!sqlite3 roman.db .tables .exit

If everything went according to plan, you should see some `[info]` lines that explain that the .csv files were converted into tables, and that you now have a database at `roman.db`. You will also see a bunch of sql commands like `CREATE TABLE IF NOT EXISTS` and these confirm that yup your materials were correctly made. The last command also confirms what `tables` exist in the database; you'll see 

`_source_info_  amphi          aqua ` 

if all went well. Notice also that `roman.db` is in your file tray now. If you double-click on that, you'll get an error because the workbench doesn't automatically know how to interpret .db files. For that, we write queries!

## Querying the Data

The first thing we're going to do is create a python function that opens a connection to the database, and allows us to build queries. After we create the function, we can create query objects, and then `run_query`. 

In [ ]:
# create a function for querying the database

# first we get the python packages that help us with this work
import sqlite3
import pandas as pd #we're giving 'pandas' a knickname so we don't always have to type 'pandas'

# we create a variable called 'db' and we use the 'sqlite' package's 'connect' function to read 'roman.db'
db = sqlite3.connect('roman.db') # this 'opens' a connection to the database. Eventually, we will close the connection.

# now we define a function called 'run query'
# that will let us write our questions using SQL syntax - the query - and then the function
# will implement it.
def run_query(query):
    return pd.read_sql_query(query,db)

Let's give it a try. We're going to build a query that asks, 'show us every column in the amphi table, but only for the first five rows.' The SQL for this:

```SELECT *``` = select everything

```FROM amphi``` = from the amphi table (which originally was amphi.csv, remember)

```LIMIT 5``` = the first five rows.

In [ ]:
query1 = 'SELECT * FROM amphi LIMIT 5;'
run_query(query1)

Notice, by the way, that the _first_ row starts with 0! Computers, eh? Indices start with row 0. Just one of those things.

Now: do the same thing again, but from the aqueducts table.

In [ ]:
## hint - copy the code from the previous code cell. Which little bit do you need to change?


### Basic Query Commands
SELECT, LIMIT, ORDER BY : using these, we can ask, 'Which amphitheatre is at the highest elevation?'

Use SELECT to retrieve the id and elevation columns FROM the amphi table

Use ORDER BY to sort the elevation column and use the DESC keyword to specify that you want to sort in descending order

Use LIMIT to restrict the output to 1 row

In [ ]:
query2 = '''
SELECT id, elevation 
FROM amphi
ORDER BY elevation DESC
LIMIT 1;
'''

run_query(query2)

## before you run this: what do you expect you'll see/find out?

In [ ]:
# Do the same thing again, but let's get the 10 amphitheatres at the highest elevation. 
# You only need to make one minimal change. Copy the code into this cell, make the change, and run it.



### Querying with Conditions

Now let's create a query that creates a subset of data using a logical operator. We need the 'WHERE' command. Our next query will look for amphitheatres that exist at an elevation greater than 500 metres above sea level.

In [ ]:
query3 = '''
SELECT * 
FROM amphi
WHERE elevation > 500;
'''

run_query(query3)

Our condition can be string data (ie, text) too; in which case we put the string in quotation marks. Here, we'll filter by the chronological grouping 'Flavian', which is to say, amphitheatres built during the reign of the Flavian Emperors (Vespasian, Titus, and Domitian, so middle of the 1st century).

In [ ]:
query4 = '''
SELECT * 
FROM amphi
WHERE chronogroup = "flavian";
'''

run_query(query4)

In [ ]:
## Can you write a query that pulls only the Flavian amphitheatres in France? 
## Hint: you'll need the AND command.

### Adding some math

How many such amphitheatres are there? This is where you'd use the COUNT command. Let's count up the number of amphitheatres from the second century. 

(By the way, this query will only give us one row of results, right? So when it runs, remember, it is showing us a table of results where there are two columns, the first without a header, and the second showing what we SELECTed. And one row of results will have an index of 0. You can ignore that.)


In [ ]:
query5 = '''
SELECT COUNT(*)
FROM amphi
WHERE chronogroup = "second-century";
'''

run_query(query5)

In [ ]:
# It might be nice to have our results show with a nice title, so:
query = '''
SELECT COUNT(*) AS "Total Count of Second Century Amphitheatres in the DB"
FROM amphi
WHERE chronogroup = "second-century";
'''

run_query(query)

We can do more than count. We can sum (add); we can find the avg (mean); we can find the min (smallest) and max (largest) values with:

SUM, AVG, MIN and MAX

Given that, what was the average capacity (in terms of number of people who could sit in the amphitheatre)?


In [ ]:
query6 = '''
SELECT AVG(capacity) AS "Average Capacity"
FROM amphi;
'''

run_query(query6)

In [ ]:

# how long were amphitheatres?

query7 = '''
SELECT AVG(arenamajor) AS "Average Length"
FROM amphi;
'''

run_query(query7)

We can group rows by one value versus another to see how they compare. Is there a difference in average length of the long axis in Julio-Claudian versus Flavian amphitheatres?

In [ ]:
query8 = '''
SELECT chronogroup, AVG(arenamajor) AS "Average Length"
FROM amphi
GROUP BY chronogroup
ORDER BY "Average Length" DESC;
'''

run_query(query8)

## Practice

Examine the fields in the aqueduct data. Craft some queries that explore that data. What kinds of questions might you want to ask?

In [ ]:
# here, let me start you off...

query10 = '''
SELECT 
FROM aqua

'''

run_query(query10)

## Joining Data
Now let's tell the database how the two tables are joined together. In this case, we want to find places where both an amphitheatre and an aqueduct existed. Let's find out if Rome has any aqueducts and amphitheatres.

Normally, when we join two tables, we want to perform the join on columns that are keyed together. In a sales database for instance there might be a table of `orders` and another for `shipping address`, and each one contains a `customer_id` column. In such a case, we use `=` to say:

```
FROM orders
INNER JOIN shipping_address
ON orders.customer_id = shipping_address.customer_id

```
Ie, 'show us orders where the customer id is the same in the orders table and the shipping address table'. In this way, our tables only ever contain one kind of information (and the relevant metadata that describes it), and we use queries to retrieve things we need (the shipping address for order #1234 which was made by customer #sg421 and whose address is ...).

But archaeological data is rarely so straightforward. In our two tables here, we have to pattern match in order to make the two fields join up - there is no 'primary key' to help us know that a row in one table is talking about the same thing in another table.

> This isn't ideal, but it's the data we've got.

Instead of `=` we're going to use the LIKE command. LIKE uses two different kinds of wildcards, `%` and `_`.

* % matches any sequence of zero or more characters
* _ matches any single character.

If we said, `LIKE 'Rom%'` we would find matches on Rome, Roman, Romania and so on. Placing the `%` on either side would find strings that *contain* Rom.

In our specific case, we want to find records where both tables contain references to "Rome" - that is, where `aqua.alternative_name` contains "Rome" AND `amphi.title` also contains "Rome". This will show us places where both an aqueduct and an amphitheatre existed in the same location (Rome).

To join our tables based on this shared location criterion, we do an 'inner join'. The syntax generally is:

```
SELECT relevant-columns                    # these will be the columns displayed in your result
FROM tableA                               # the table to join
INNER JOIN tableB                         # with this table
ON condition1 AND condition2              # by these criteria
```

The query below finds records where both the `aqua.alternative_name` and `amphi.title` fields contain the string "Rome", effectively showing us the aqueducts and amphitheatres that existed in Rome.


In [ ]:

query11 = '''
SELECT amphi.id, amphi.label, amphi.title, aqua.aqueduct_name, aqua.alternative_name
FROM aqua 
INNER JOIN amphi
ON LOWER(aqua.alternative_name) LIKE '%rome%' 
   AND LOWER(amphi.title) LIKE '%rome%';
'''
run_query(query11)




In [ ]:
# Take a look at the csv files, and see if you can repurpose the code for query11 for some other location. 
# Arles is nice this time of year I hear... but note which field that name appears in...

query12 = '''

'''

## Updating the database with new data?

SQLITE has a wide variety of commands - you can search for these easily enough - but I thought it'd be useful to show you how to update a table with new data, or to remove data. Back at the beginning, we defined something called `db` and connected it to the database. If you look up tutorials about sqlite, they'll talk about the 'cursor object' and how it allows you to perform various database operations. For us, `db` is that cursor object. (For more on sqlite, [this seems handy](https://www.sqlitetutorial.net/)).

To update the database tables, we just get that cursor object to insert a new row or delete it and so on.

In [ ]:
# Insert data into the table
db.execute("INSERT INTO amphi (id, title, label) VALUES ('ForumNovumAmphitheatre', 'Amphitheatre at Forum Novum', 'Forum Novum')")
# the next command makes the change permanent:
db.commit()

# now let's take a look
query13 = 'SELECT * FROM amphi;'
run_query(query13)

# scroll to the end and you'll see our new data about the amphitheatre at Forum Novum!

In [ ]:
# Delete data from the table
db.execute("DELETE FROM amphi where id = 'ForumNovumAmphitheatre'")
db.commit
query14 = 'SELECT * FROM amphi;'
run_query(query14)

There's a lot more to sql and sqlite than this. But this will get you started.

# Visualizing The Results

By putting our csv files of raw data into an sqlite database, we are able to run various queries. Because we are working in a Python environment, we can leverage various data visualization libraries to see our data.

Let's say we're investigating a potential hypothesis about where the Romans would situate an amphitheatre. Maybe our idea is that the Romans really _appreciated_ panoramic views as part of their entertainment. Because panoramas tend to get better the higher you go, amphitheatres that _were_ higher were more popular, and so were built with greater capacity to handle the crowd. So we might begin to explore this idea by looking for any sort of relationship between elevation and capacity.

...I'm not saying that Romans did this or that the logic is watertight; just that this is an idea that could be explored. (A more polished exploration would look for clustering, or viewsheds, etc, and the kinds of things we did when we were talking about spatial analysis).

We query the database for the information; we double check the data types to make sure we've got _values_ instead of _strings (characters)_; we use some plotting packages to work with the information.

In [ ]:
import matplotlib.pyplot as plt
# now we run a query; but unlike in the previous notebook, we'll put all of the SQL commands on a single line
# the line below defines a dataframe, 'df', and uses the pandas command pd.read_sql_query to run a query on the database connection we defined above, db.
query15 = """
SELECT id, elevation, capacity FROM amphi ORDER BY elevation DESC
"""
df = pd.read_sql_query(query15, db) # we use the 'read_sql_query' function in pandas, 
                        # telling it what the query is and what database to apply it to
                        # and then instead of printing out the result, it turns it into a table of data
            
df # and this lets us check the result.


Do you see a problem under the 'capacity' column? Lots of empty cells. We have to tell Python how to deal with that. 

Pandas expects missing values to be coded NaN rather than just be empty cells. Otherwise we'll get errors. Also, if we're thinking numbers=something numeric, that's not always the case. So first we'll check, and then we'll fix.



In [ ]:
# We need to check to see whether or not our data is actually numeric
# If there are no columns of a given type, then the result will just print empty [ ] 
print("String columns:", df.select_dtypes(include=['object', 'string']).columns.tolist())
print("Numeric columns:", df.select_dtypes(include=['number']).columns.tolist())
print("DateTime columns:", df.select_dtypes(include=['datetime']).columns.tolist())

We can only plot those columns that are numeric. Right now, the data under 'capacity' might as well be a label, as far as python cares. That is to say, it just regards that information as strings of characters rather than values.

In [ ]:
# We covert the column to the numeric data type.
df['capacity'] = df['capacity'].apply(pd.to_numeric, errors='coerce')

In [ ]:
# compare the results here with the first time we looked. What's different?
# we can look inside a dataframe by giving the name of the dataframe (here, 'df') and the
# index positions for the rows we want: row 1 to row 5. NB! rows 1 to 5 are index 0 to 4.
# Computers, eh? Drives you nuts.

df[0:5]

In [ ]:
# Now we can begin to plot data.
# to plot things with pandas, we also need a package called matplotlib
%matplotlib inline
import matplotlib.pyplot as plt



# here's how we make a simple scatter plot
# For this demo, we'll create two variables and give them a bunch of values:

X = [590,540,740,130,810,300,320,230,470,620,770,250]
Y = [32,36,39,52,61,72,77,75,68,57,48,48]

# then we'll use the 'scatter' function from matplotlib.pylot via its alias, 'plt' to plot this data:
plt.scatter(X,Y)
plt.show()

# you will be able to find lots of documentation online for the various kinds of things matplotlib can do.

In [ ]:
# Now that you know the pattern, let's plot elevation against capacity. 

plt.scatter(df['elevation'], df['capacity']) #<- notice how we pass the elevation and capacity columns as x and y
plt.show()

In [ ]:
# Let's make the plot pretty

plt.scatter(df['elevation'], df['capacity'])
#add title
plt.title('Relationship Between Elevation and Capacity in Roman Amphitheatres')

#add x and y labels
plt.xlabel('Elevation')
plt.ylabel('Capacity')


In [ ]:
# let's print the plot to raster and to vector image
# solution courtesy https://stackoverflow.com/questions/9622163/save-plot-to-image-file-instead-of-displaying-it-using-matplotlib

fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
ax.scatter(df['elevation'], df['capacity']) 
plt.title('Relationship Between Elevation and Capacity \n in Roman Amphitheatres') # the \n is an escape character that tells the machine to insert a newline
plt.ylabel('Capacity')
plt.xlabel('Elevation')
fig.savefig('demo.png')   # save the figure to file for raster image
fig.savefig('demo.pdf') # save the figure to file for vector image
plt.close(fig)    # when you make a fig, the plt commands are all adding elements to it; you need to close the fig when you've finished.


...and here's how you embed an image into a markdown document: ```![alt text description of the image](demo.png)```

![](demo.png)

In [ ]:
## and when you're finished, you always should 
# close the connection to the database:
db.close()

# Your Turn
Now - go to the [[graveyard-data-as-database.ipynb]] and see what you can achieve with some graveyard data collected by students.